In [ ]:
import pandas as pd
import numpy as np
import math

In [ ]:
df_pitch_subs = pd.read_csv('pitching_subs.csv')
df_pitch_subs.head()

In [ ]:
def save_sit(row):
    if row['BAT_HOME_ID'] == 0 and row['HOME_SCORE_CT'] > row['AWAY_SCORE_CT']:
        return True
    elif row['BAT_HOME_ID'] == 1 and row['HOME_SCORE_CT'] < row['AWAY_SCORE_CT']:
        return True
    else:
        return False

df_pitch_subs['SAVE_SIT'] = df_pitch_subs.apply(save_sit, axis=1)

In [ ]:
df_pitch_subs[['GAME_ID', 'PIT_ID', 'INN_CT', 'BAT_HOME_ID', 'AWAY_SCORE_CT', 'HOME_SCORE_CT', 'SAVE_PIT_ID', 'SAVE_SIT']].head(100)

In [ ]:
df_pitch_subs = df_pitch_subs[df_pitch_subs['SAVE_SIT'] == True].reset_index()

In [ ]:
df_pitch_subs[['GAME_ID', 'PIT_ID', 'INN_CT', 'BAT_HOME_ID', 'AWAY_SCORE_CT', 'HOME_SCORE_CT', 'SAVE_PIT_ID', 'SAVE_SIT']].head(10)

In [ ]:
df_next_pitcher = df_pitch_subs.shift(-1).reset_index()
df_next_pitcher[['GAME_ID', 'PIT_ID', 'INN_CT', 'BAT_HOME_ID', 'AWAY_SCORE_CT', 'HOME_SCORE_CT', 'SAVE_PIT_ID', 'SAVE_SIT']].head(100)

In [ ]:
df_pitch_subs['made_save'] = None
df_pitch_subs.head()

In [ ]:
print(len(df_pitch_subs))
print(len(df_next_pitcher))
for t1, t2 in zip(df_pitch_subs.iterrows(), df_next_pitcher.iterrows()):
    cur_index = t1[0]
    cur_row = t1[1]
    next_row = t2[1]
    made_save = None
    
    if cur_row['GAME_ID'] == next_row['GAME_ID'] and cur_row['BAT_HOME_ID'] == next_row['BAT_HOME_ID']:
        # in same game, and same team.
        if cur_row['BAT_HOME_ID'] == 0:
            # home team
            blew_lead = next_row['HOME_SCORE_CT'] <= next_row['AWAY_SCORE_CT']
            if blew_lead:
                print('H1')
                inn_diff = next_row['INN_CT'] - cur_row['INN_CT']
                init_score_diff = cur_row['HOME_SCORE_CT'] - cur_row['AWAY_SCORE_CT']
                
                if (init_score_diff <= 3 and inn_diff >= 1) or (inn_diff >= 3) or (cur_row['HOME_SCORE_CT'] <= cur_row['AWAY_SCORE_CT'] + 2 + sum([cur_row['onFirst'], cur_row['onSecond'], cur_row['onThird']])):
                    made_save = False
                
            
        elif cur_row['BAT_HOME_ID'] == 1:
            # away team
            blew_lead = next_row['HOME_SCORE_CT'] >= next_row['AWAY_SCORE_CT']
            if blew_lead:
                print('H2')
                inn_diff = next_row['INN_CT'] - cur_row['INN_CT']
                init_score_diff = cur_row['AWAY_SCORE_CT'] - cur_row['HOME_SCORE_CT']
                
                if (init_score_diff <= 3 and inn_diff >= 1) or (inn_diff >= 3) or (cur_row['AWAY_SCORE_CT'] <= cur_row['HOME_SCORE_CT'] + 2 + sum([cur_row['onFirst'], cur_row['onSecond'], cur_row['onThird']])):
                    made_save = False
    
    
    elif cur_row['PIT_ID'] == cur_row['SAVE_PIT_ID']:
            made_save = True
               

    df_pitch_subs.iloc[cur_index, df_pitch_subs.columns.get_loc('made_save')] = made_save
    
    
df_pitch_subs.head(50)

In [ ]:
df_blown_save = df_pitch_subs[df_pitch_subs['made_save'] == False]
df_blown_save.head()